In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import itertools
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

まずは導入\
「esc + h」でショートカットの確認\
「esc + m」でマークダウンに変更\
「esc + y」でコードに変更

In [4]:
train = pd.read_csv('../input/ch01-titanic/train.csv')
test = pd.read_csv('../input/ch01-titanic/test.csv')

学習データ、テストデータの読み込みを行なっている。

In [5]:
train_x = train.drop(['Survived'], axis = 1)
train_y = train['Survived']

学習データは特徴量と目的変数に分けている。

In [6]:
test_x = test.copy()

テストデータは特徴量のみなのでそのままコピーしている。
pythonのコピーについて「deepcopy」とかいうのがあるらしくcopyとの違いがわからないので参考urlを貼っておく。
https://murashun.jp/article/programming/python/python-list-copy-deepcopy.html

# 特徴量作成

In [7]:
train_x = train_x.drop(['PassengerId'], axis = 1)
test_x = test_x.drop(['PassengerId'], axis = 1)

変数PassengerIdを除外している。

In [8]:
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis = 1)

変数Name, Ticket, Cabinを除外する。

In [9]:
for c in ['Sex', 'Embarked']:
    # 学習データに基づいてどう変換するかを定める
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))
    
    # 学習データ、テストデータを変換する
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))

それぞれのカテゴリ変数にlabel encodeingを適用している。

Label Encoder:　データ（特にラベル、質的データについて数値に変換する）
le.fit(data):学習
le.transform(data):適応

説明url:https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

# モデル作成

In [10]:
model = XGBClassifier(n_estimators = 20, random_state = 71)
model.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=20, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=71,
              reg_alpha=0, reg_lambda=1, ...)

モデルの生成および学習データの予測値を確率で出力する。

In [11]:
pred = model.predict_proba(test_x)[:, 1]

テストデータ予測値を確率で出力する。

In [12]:
pred_label = np.where(pred > 0.5, 1, 0)

テストデータの予測値を二値に変換する。

In [13]:
submission = pd.DataFrame(
    {'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index = False)

スコア:0.7799(本書中の数値と異なる可能性がある)

# バリデーション

In [14]:
scores_accuracy = []
scores_logloss = []

各foldのスコアを保存するリスト

In [15]:
kf = KFold(n_splits = 4, shuffle = True, random_state = 71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    #モデルの学習を行う
    model = XGBClassifier(n_estimators = 20, random_state = 71)
    model.fit(tr_x, tr_y)
    
    #バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]
    
    #バリデーションデータでのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    
    #そのfoldのスコアを保存する
    scores_logloss.append(accuracy)
    scores_accuracy.append(accuracy)

クロスバリデーションを行う。
学習データを４つに分割し、うち１つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す。

In [16]:
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss:{logloss:.4f}, accuracy: {accuracy:.4f}')

logloss:0.8182, accuracy: 0.8182


# モデルチューニング

In [17]:
param_space = {
    'max_depth':[3, 5, 7],
    'min_child_weight':[1.0, 2.0, 4.0]
}

チューニング候補とするパラメータを準備する

In [18]:
param_combinations = itertools.product(
    param_space['max_depth'], param_space['min_child_weight'])

探索するハイパーパラメータの組み合わせ

In [19]:
params = []
scores = []

各パラメータの組み合わせ、それに対するスコアを保存するリスト

In [20]:
for max_depth, min_child_weight in param_combinations:
    score_folds = []
    #クロスバリデーションを行う
    #学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す。
    
    kf = KFold(n_splits = 4, shuffle = True, random_state = 123456)
    for tr_idx, va_idx in kf.split(train_x):
        #学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        
        #モデルの学習を行う
        model = XGBClassifier(n_estimators = 20, random_state = 71,
                              max_depth = max_depth, min_child_weight = min_child_weight)
        model.fit(tr_x, tr_y)
        
        #バリデーションデータでのスコアを計算し、保存する。
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)
        
    #各foldのスコアを平均する
    score_mean = np.mean(score_folds)
    
    #バラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

各パラメータの組み合わせごとに、クロスバリデーションで評価を行う。出力された結果が最適のmax_depthとmin_child_weight

In [22]:
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

max_depth: 3, min_child_weight: 2.0


# ロジスティック回帰用の特徴量の作成

In [24]:
train_x2 = train.drop(['Survived'], axis = 1)
text_x2 = test.copy()

元データをコピーする。